In [4]:
PROMPT = """You are an ai trainer in marketplace company. You need to create a JSON from a plain text of a product offer provided by user. This JSON must contain next fields:
"place" - where buyer can recieve product (or delivery info)
"products" - list of dictionaries of valid products. 
Each product dict must contain ONLY following information: 
"name" - short title for a customer, 
"price" - cost of one unit of a product (one number), 
"currency" - in what currency is the price indicated (default is 'рубли'),
"size" - size or weight of a product (one string),
"count" - number of product units offered.
Represent different variations on each product as DIFFERENT products.
Do NOT add description and any other fields. Drop all fields with null value.
If NO information about the price provided, the product has to be excluded from the answer.
Answer ONLY with JSON string, no other text is needed.
"""
API_KEY = ''

In [5]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable


In [25]:
import openai
import pickle as pkl
import pandas as pd
import json
from tqdm.notebook import tqdm

In [6]:
openai_client = openai.OpenAI(api_key=API_KEY)

In [17]:
with open('../cache/vk_filtered_1500.pkl', 'rb') as f:
  df = pkl.load(f)

In [11]:
import requests
response = requests.get('https://example.com')
response.status_code

200

In [26]:
from time import sleep

class OpenAIShooter:
  def __init__(self, client, prompt):
    self.client = client
    self.cache = {}
    self.model = "gpt-3.5-turbo"
    self.prompt = prompt

  def request(self, text):
    if text not in self.storage:
      retry = True
      while retry:
        response = self.client.chat.completions.create(
          model=self.model,
          messages=[
            {"role": "system", "content": self.prompt},
            {"role": "user", "content": text}
          ],
          **({'response_format':{"type": "json_object"}} if self.model == "gpt-3.5-turbo-1106" else {})
        )
        self.storage[text] = response
        retry = response.status_code != 200
        if retry:
          sleep(0.5)
    return self.storage[text]
  
  def shoot_all(self, df):
    answer = {}
    for id, text in df.text.items():
      try:
        answer[id] = json.loads(self.request(text))
      except:
        answer[id] = self.request(text)


In [27]:
shooter = OpenAIShooter(openai_client, PROMPT)

In [ ]:
shooter.shoot_all(df)